# About This Notebook

This notebook was done on Kaggle and downloaded after. It contains code that we use to scrape the games data from the 20k to 30k rows of our list of games. 

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/steam60kgamesdata/gamesdata.csv


In [2]:
df = pd.read_csv('/kaggle/input/steam60kgamesdata/gamesdata.csv')

In [3]:
from bs4 import BeautifulSoup
import time
import requests
import json
from tqdm import tqdm

In [4]:
counter = 0
data = []

for counter in tqdm(range(10000,20000)):
        hlink = df['Link'][counter]
        response = requests.get(hlink)
        soup1 = BeautifulSoup(response.text, 'html.parser')

        description = soup1.find('div',id='game_area_description')
        description = description.text.strip().replace('\r','').replace('\t','').replace('\n','') if description else None

        reviews = soup1.find('div', class_='user_reviews_summary_row')
        reviews = reviews.text.strip().replace('\r','').replace('\t','').replace('\n','') if reviews else None
        tags_div = soup1.find('div', class_='glance_tags popular_tags')
        tags = tags_div.find_all('a', class_='app_tag') if tags_div else 'N/A'

        if tags != 'N/A':
            tagged = []
            for tag in tags:
                tagged.append(tag.text.strip())
        else:
            tagged = 'N/A'

        tagged_string = ', '.join(tagged)
        data.append([description, reviews, tagged_string])
        
        counter=+1
    

100%|██████████| 10000/10000 [2:32:04<00:00,  1.10it/s]


In [5]:
data_df = pd.DataFrame(data, columns=['description','reviews','tags'])
data_df

,description,reviews,tags
0,About This GameThe Planet After Us computer ga...,All Reviews:1 user reviews- Need more user rev...,"Adventure, Shooter, FPS, 3D, Relaxing, Open Wo..."
1,About This GameNORCO is a Southern Gothic poin...,Recent Reviews:Very Positive(121)- 88% of the ...,"Point & Click, Mystery, Story Rich, Atmospheri..."
2,About This GameBuild a medieval village within...,All Reviews:Mostly Positive(263)- 79% of the 2...,"Simulation, Strategy, VR, Casual, Indie, City ..."
3,"About This GameIn this elegy of blood, Athens ...",All Reviews:Mixed(100)- 40% of the 100 user re...,"Casual, RPG, Interactive Fiction, Vampire, Act..."
4,About This GameGale and Arthur have been datin...,All Reviews:Positive(12)- 100% of the 12 user ...,"Casual, Dating Sim, Interactive Fiction, Visua..."
...,...,...,...
9995,About This GameThe story is set in a dystopic ...,All Reviews:1 user reviews- Need more user rev...,"Casual, Action-Adventure, Puzzle Platformer, F..."
9996,About This GameA short match 3 game with uniqu...,All Reviews:1 user reviews- Need more user rev...,"Casual, Adventure, Strategy, Arcade, 2D, Logic..."
9997,About This GameOpa Definitive Edition is a 2D ...,All Reviews:1 user reviews- Need more user rev...,"Casual, Arcade, 2D, 3D, Cartoony, Hand-drawn, ..."
9998,"About This GamePOG - cute, minimal physical pu...",All Reviews:Mostly Positive(14)- 71% of the 14...,"Casual, Puzzle, Abstract, Indie, Cute, Singlep..."


In [6]:
data_df.to_csv('thirty_k_games.csv', index=False)